In [1]:
import sys
sys.path.insert(0, '../src/')

import pandas as pd
import kaggle
import auto_co2 as co2
import importlib
importlib.reload(co2)

#auth_file_path = ''
#df = co2.data.download_and_load_co2_data()
df = co2.data.load_co2_data(filepath='../../data/raw')
df = co2.data.dataviz_preprocess(df)
df['ElectricRange'] = df['ElectricRange'].fillna(0)
#df['InnovativeEmissionsReductionWltp'] = df['InnovativeEmissionsReductionWltp'].fillna(0)
#df['ElectricConsumption'] = df['ElectricConsumption'].fillna(0)
df = co2.data.standardize_innovtech(df)
df = df.drop(columns=['ID', 'Country', 'ManufacturerName', 'WltpTestMass', 'FuelConsumption', 'RegistrationDate', 'AxleWidthSteering', 'ManufNameOem', 'FuelMode', 'EngineCapacity', 'AxleWidthSteering'])


df = co2.data.conditional_column_update(df, 'CategoryOf', 'm1', 'CategoryOf', 'M1')

# Group by the specified columns and compute the mean for the other columns


2024-01-15 21:56:08.095048: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-15 21:56:08.111095: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Incomplete rows dropped:473981


In [2]:
counts = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology', ]).size()

# Group by the specified columns and compute the mean for the other columns
df = df.groupby(['CommercialName', 'FuelType', 'CategoryOf', 'InnovativeTechnology', ]).agg({
    'Pool': lambda x: x.mode()[0],
    'Co2EmissionsWltp': 'mean',
    'EnginePower': 'mean',
    #'ElectricConsumption': 'mean',
    #'InnovativeEmissionsReductionWltp': 'mean',
    'ElectricRange': 'mean',
    'MassRunningOrder': 'mean',
    'BaseWheel': 'mean'
}).reset_index()

# Add the 'Count' column to the aggregated DataFrame
df['Count'] = counts.values
df['Share'] = df['Count'] / df['Count'].sum()
df.sort_values(by=['Count'], ascending=False)

display(df.head(10))



,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Pool,Co2EmissionsWltp,EnginePower,ElectricRange,MassRunningOrder,BaseWheel,Count,Share
0,---,PETROL,M1,0,STELLANTIS,132.000000,96.0,0.0,1323.000000,2605.0,1,1.058635e-07
1,108,PETROL,M1,0,STELLANTIS,110.441662,53.0,0.0,921.120371,2340.0,30863,3.267265e-03
2,116 / / D,DIESEL,M1,0,BMW,120.000000,85.0,0.0,1450.000000,2670.0,3,3.175905e-07
3,116 / / D,DIESEL,M1,1,BMW,119.500000,85.0,0.0,1450.000000,2670.0,4,4.234540e-07
4,116 / / D AUT.,DIESEL,M1,0,BMW,121.000000,85.0,0.0,1460.000000,2670.0,3,3.175905e-07
5,116 / / D AUT.,DIESEL,M1,1,BMW,120.600000,85.0,0.0,1460.000000,2670.0,5,5.293175e-07
6,116 / / I,PETROL,M1,1,BMW,130.625000,80.0,0.0,1395.000000,2670.0,8,8.469079e-07
7,116 / / I AUT.,PETROL,M1,1,BMW,136.000000,80.0,0.0,1415.000000,2670.0,7,7.410445e-07
8,116 I,PETROL,M1,1,BMW,132.000000,80.0,0.0,1850.000000,2670.0,1,1.058635e-07
9,116D,DIESEL,M1,0,BMW,121.660680,85.0,0.0,1456.757282,2670.0,2060,2.180788e-04


In [4]:
df = co2.data.dummify_all_categoricals(df, dummy_columns=['Pool', 'FuelType', 'CategoryOf'], should_discretize_electricrange=False)

co2.data.save_processed_data(df, classification=False, pickle=False)

Data saved to ../data/processed/co2_regression_20240115-215812


In [3]:
df.sort_values(by=['Count'], ascending=False)   

,CommercialName,FuelType,CategoryOf,InnovativeTechnology,Pool,Co2EmissionsWltp,EnginePower,ElectricRange,MassRunningOrder,BaseWheel,Count,Share
2351,CLIO,PETROL,M1,1,RENAULT-NISSAN-MITSUBISHI,116.015614,66.192488,0.0,1181.066234,2582.949597,128922,1.364813e-02
6338,PANDA,PETROL,M1,0,STELLANTIS,122.401628,52.846083,0.0,1069.384510,2300.005183,108247,1.145941e-02
89,208,PETROL,M1,1,STELLANTIS,119.691755,69.166199,0.0,1133.583302,2540.000304,98623,1.044058e-02
1873,C3,PETROL,M1,1,STELLANTIS,125.206536,66.864953,0.0,1101.554765,2542.657380,93427,9.890509e-03
7829,T-ROC,PETROL,M1,1,VW-SAIC,147.736943,100.461553,0.0,1364.432126,2596.325762,93246,9.871347e-03
...,...,...,...,...,...,...,...,...,...,...,...,...
6443,PORSCHE TAYCAN TURBO,ELECTRIC,M1,0,VW-SAIC,0.000000,460.000000,0.0,2380.000000,2900.000000,1,1.058635e-07
6470,PROACE CITY,DIESEL,M1,0,MAZDA-SUBARU-SUZUKI-TOYOTA,148.000000,96.000000,0.0,1615.000000,2975.000000,1,1.058635e-07
25,118 / / I,PETROL,M1,0,BMW,137.000000,103.000000,0.0,1365.000000,2670.000000,1,1.058635e-07
29,118 D,DIESEL,M1,0,BMW,132.000000,110.000000,0.0,1505.000000,2670.000000,1,1.058635e-07
